# This notebook is colab compatible.

In [13]:
# connect to google drive and proceed

!mkdir /content/drive/MyDrive/code_assistant && cd /content/drive/MyDrive/code_assistant

In [ ]:
# Installing ollama

!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 593.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00


In [ ]:
%%writefile Modelfile

FROM codellama

## Set the temperature
PARAMETER temperature 1

## set the system prompt
SYSTEM """
You are a code teaching assistant. You will give response to the user as a teacher who will explain the code sections
to make them more understandable.
"""

Overwriting Modelfile


In [ ]:
%%writefile ollama_serve.py

import subprocess

p = subprocess.Popen(["ollama", "serve"])

print(p.pid)

Overwriting ollama_serve.py


In [10]:
!nohup python ./ollama_serve.py &

nohup: appending output to 'nohup.out'


In [3]:
!ollama create codementor -f Modelfile

transferring model data 
using existing layer sha256:3a43f93b78ec50f7c4e4dc8bd1cb3fff5a900e7d574c51a6f7495e48486e0dac 
using existing layer sha256:8c17c2ebb0ea011be9981cc3922db8ca8fa61e828c5d3f44cb6ae342bf80460b 
using existing layer sha256:590d74a5569b8a20eb2a8b0aa869d1d1d3faf6a7fdda1955ae827073c7f502fc 
using existing layer sha256:2e0493f67d0c8c9c68a8aeacdf6a38a2151cb3c4c1d42accf296e19810527988 
using existing layer sha256:6d8aa47b10cdb6baf882e78a0238dcbfbd1ede796386ec94a3970f32bf7c5832 
using existing layer sha256:c49a7d8d17448912729269cadf0325a4cd4e6e7a4fdd43d8b7c35d5f6ca23861 
using existing layer sha256:938e6c20cac1eda3edee99c198ae829440eb6d8a847c82b2cb861efd4b3aa26f 
writing manifest 
success 


In [ ]:
import requests
import json
import gradio as gr

url = "http://localhost:11434/api/generate"

headers = {
    'Content-Type': 'application/json'
}

history = []

def generate_response(prompt):

  history.append(prompt)

  final_prompt = "\n".join(history)

  data = {
      "model": "codementor",
      "prompt": final_prompt,
      "stream": False
  }

  response = requests.post(url, headers = headers, data=json.dumps(data))

  if response.status_code == 200:
    response = response.text
    data = json.loads(response)
    actual_response = data['response']
    return actual_response

  else:
    print("error: ", response.text)

interface = gr.Interface(
    fn = generate_response,
    inputs = gr.Textbox(lines=4, placeholder="Enter your prompt"),
    outputs = "text"
)

interface.launch()